# Imports

In [36]:
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Model load

In [37]:
path_qing_cheng = '../model/qing_cheng'
path_vallen = '../model/vallen'

qing_cheng_model_name = 'Qing-Cheng'
vallen_model_name = 'Vallen'

In [38]:
def get_file_paths(path: str):
    file_paths = []

    for dir_path, _, file_names in os.walk(path):
        file_path = {}
        
        for file_name in file_names:
            
            if file_name.endswith('.h5'):
                file_path['model'] = os.path.join(dir_path, file_name)
                
            elif file_name.endswith('x.npy'):
                file_path['x'] = os.path.join(dir_path, file_name)
                
            elif file_name.endswith('y.npy'):
                file_path['y'] = os.path.join(dir_path, file_name)
                
        if len(file_path) > 0:
            file_paths.append(file_path)

    return file_paths

In [39]:
paths_qing_cheng = get_file_paths(path_qing_cheng)
paths_vallen = get_file_paths(path_vallen)

In [42]:
def load_model_result(path_dict: dict):
    model = load_model(path_dict['model'])
    x = np.load(path_dict['x'])
    y = np.load(path_dict['y'])
    
    y_predicted = model.predict(x)
    y_predicted_classes = np.argmax(y_predicted, axis=1)
    
    return {
        'model': model, 
        'x': x, 
        'y': y, 
        'y_predicted': y_predicted, 
        'y_predicted_classes': y_predicted_classes,
    }

In [43]:
qing_cheng_models = [load_model_result(path) for path in paths_qing_cheng]
vallen_models = [load_model_result(path) for path in paths_vallen]

# Plot

In [44]:
position_class_label = ['No Leakage', 'Leakage in position 2', 'Leakage in position 3', 'Leakage in position 4']
boolean_class_label = ['No Leakage', 'Leakage']

In [46]:
def plot_confusion_matrix(model_a:dict, model_a_name: str, model_b:dict, model_b_name:str, class_labels: list[str], title:str):
    matrix_a = confusion_matrix(model_a['y'], model_a['y_predicted_classes'])
    matrix_b = confusion_matrix(model_b['y'], model_b['y_predicted_classes'])
    
    matrix_a_percent = matrix_a / matrix_a.sum(axis=1)[:, np.newaxis] * 100
    matrix_b_percent = matrix_b / matrix_b.sum(axis=1)[:, np.newaxis] * 100
    
    matrix_a_df = pd.DataFrame(matrix_a_percent, columns=class_labels, index=class_labels)
    matrix_b_df = pd.DataFrame(matrix_b_percent, columns=class_labels, index=class_labels)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    heatmap_a = sns.heatmap(matrix_a_df, annot=True, cmap='Blues', fmt='.1f', cbar=False, square=True, linewidths=0.5, ax=ax1)
    heatmap_b = sns.heatmap(matrix_b_df, annot=True, cmap='Blues', fmt='.1f', cbar=False, square=True, linewidths=0.5, ax=ax2)
    
    # Rotate x-axis labels by 45 degrees
    heatmap_a.set_xticklabels(heatmap_a.get_xticklabels(), rotation=45, ha='right')
    heatmap_b.set_xticklabels(heatmap_b.get_xticklabels(), rotation=45, ha='right')
    
    ax1.set_title(f'{model_a_name}')
    ax2.set_title(f'{model_b_name}')
    
    plt.suptitle(title, y=1.05)
    
    plt.tight_layout()
    plt.show()

In [47]:
for index in range(len(qing_cheng_models)):
    if index < 4:
        plot_confusion_matrix(
            qing_cheng_models[index], 
            qing_cheng_model_name, 
            vallen_models[index], 
            vallen_model_name, 
            boolean_class_label, 
            'Confusion matrix: Binary state (%)'
        )
    
    else:
        plot_confusion_matrix(
            qing_cheng_models[index], 
            qing_cheng_model_name, 
            vallen_models[index], 
            vallen_model_name, 
            position_class_label,
            'Confusion matrix: Sensor position (%)'
        )